## Kaggle Competition Experience

### Airbnb New User Bookings

#### numerical data
* merge train data and test data in order to do all the following things to them together. When we need to split them for train and test, then split them.
* group data into numerical, categorical, text, datetime
* extract datetime data
* numerical data interaction, with time date, without time date
* group data

#### categorical data
* label encoding
* one hot encoding
* leave one out encoding
* group data

#### train and test
* create train and test data
* cross validation
* predict on test data


## <font color = 'blue'>Links</font>

https://www.kaggle.com/svpons/airbnb-recruiting-new-user-bookings/script-0-8655 <br />
This is a very simple code and got 0.86
<br /><br />
http://brettromero.com/wordpress/data-science-kaggle-walkthrough-cleaning-data/ <br />
This is someone telling how to clean data

https://www.kaggle.com/c/airbnb-recruiting-new-user-bookings/data <br />
The original link for this competition

## Import Packages

In [11]:
# ------------------------------ import ------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
% matplotlib inline
from sklearn import datasets
from sklearn import svm
from sklearn import cross_validation
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import neighbors
from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

## Step 0 Read Data

In [12]:
# -------------------------------- 0 read data --------------------------------
sessionRawData = pd.read_csv('sessions.csv')
countriesRawData = pd.read_csv('countries.csv')
ageGenderRawData = pd.read_csv('age_gender_bkts.csv')
trainUser2RawData = pd.read_csv('train_users_2.csv')
testUserRawData = pd.read_csv('test_users.csv')

In [13]:
# ----------------------------- merge train and test data -----------------------------
# need to merge train data and test data here
# then do the same thing to both of them
train_size = trainUser2RawData.shape[0]
test_size = testUserRawData.shape[0]

fullData = pd.concat([trainUser2RawData, testUserRawData])

In [14]:
# print trainUser2RawData.shape
# print testUserRawData.shape
# print fullData.shape
# print fullData['country_destination'].isnull().value_counts()
# ----------------------------- check -----------------------------

## Step 1 Data Wrangling and exploration

In [15]:
# -------------------------- have a look at the data --------------------------
print fullData.describe()
print '\n this is the null data in age column\n' , fullData.age.isnull().value_counts()

                 age    signup_flow  timestamp_first_active
count  158681.000000  275547.000000            2.755470e+05
mean       47.145310       4.291965            2.013310e+13
std       142.629468       8.794313            9.146438e+09
min         1.000000       0.000000            2.009032e+13
25%        28.000000       0.000000            2.013040e+13
50%        33.000000       0.000000            2.014010e+13
75%        42.000000       1.000000            2.014062e+13
max      2014.000000      25.000000            2.014093e+13

 this is the null data in age column
False    158681
True     116866
Name: age, dtype: int64


In [16]:
# ---------------------------- 1 data cleansing ----------------------------

# convert all date columns to datetime data type 
fullData['date_account_created'] = pd.to_datetime(fullData.date_account_created)
fullData.timestamp_first_active = pd.to_datetime(fullData.timestamp_first_active.astype('str'))
fullData.date_first_booking = pd.to_datetime(fullData.date_first_booking)


# replace the ones who greater than 100 years old with NaN
for ii in fullData[fullData.age > 100].age.unique():
    fullData.age.replace(to_replace = ii, value = np.nan, inplace = True)
# replace the ones who less than 14 years old with NaN 
for ii in fullData[fullData.age < 14].age.unique():
    fullData.age.replace(to_replace = ii, value = np.nan, inplace = True)
    
# print fullData[fullData.age > 100].age.value_counts() , 'the ones greater than 110 have been replaced'
# print fullData[fullData.age < 14].age.value_counts() , 'the ones less than 10 have been replaced by 0'
# print fullData[fullData.age < 0].age.value_counts() , 'so there is no one less than 0'
# print fullData.age.value_counts()


In [17]:
print fullData.age.describe()
print '\n' , fullData.age.isnull().value_counts() , '\n'
# print fullData.age.mean() , ' is mean value of age \n'
print fullData.age.median() , ' is median value of age'

count    155932.000000
mean         36.055043
std          11.663664
min          15.000000
25%          28.000000
50%          33.000000
75%          42.000000
max         100.000000
Name: age, dtype: float64

False    155932
True     119615
Name: age, dtype: int64 

33.0  is median value of age


In [18]:
# and then we find that there are these mean and median value of age
# I have tried replace the NaN with very big number
# now it's the time I replace them with the average value or median value

In [19]:
# replace outliers with -1
fullData.age.replace(to_replace = np.nan, value = -1, inplace = True)
print fullData.age.describe()
print '\n' , fullData.age.isnull().value_counts() , '\n'

count    275547.000000
mean         19.969443
std          20.354181
min          -1.000000
25%          -1.000000
50%          25.000000
75%          34.000000
max         100.000000
Name: age, dtype: float64

False    275547
Name: age, dtype: int64 



In [20]:
fullData.head(2)

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,date_first_booking,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
0,direct,direct,-1.0,NDF,2010-06-28,NaT,untracked,Chrome,Mac Desktop,-unknown-,gxn3p5htnn,en,Web,0,facebook,2009-03-19 04:32:55
1,seo,google,38.0,NDF,2011-05-25,NaT,untracked,Chrome,Mac Desktop,MALE,820tgsjxq7,en,Web,0,facebook,2009-05-23 17:48:09


### check the null value in each columns

In [21]:
# print fullData.date_first_booking.head(2)
# xxx = fullData.date_first_booking.dt.year
# print xxx.head(2)
for header in fullData:
    print header , '\n' , fullData[header].isnull().value_counts() , '\n'

affiliate_channel 
False    275547
Name: affiliate_channel, dtype: int64 

affiliate_provider 
False    275547
Name: affiliate_provider, dtype: int64 

age 
False    275547
Name: age, dtype: int64 

country_destination 
False    213451
True      62096
Name: country_destination, dtype: int64 

date_account_created 
False    275547
Name: date_account_created, dtype: int64 

date_first_booking 
True     186639
False     88908
Name: date_first_booking, dtype: int64 

first_affiliate_tracked 
False    269462
True       6085
Name: first_affiliate_tracked, dtype: int64 

first_browser 
False    275547
Name: first_browser, dtype: int64 

first_device_type 
False    275547
Name: first_device_type, dtype: int64 

gender 
False    275547
Name: gender, dtype: int64 

id 
False    275547
Name: id, dtype: int64 

language 
False    275547
Name: language, dtype: int64 

signup_app 
False    275547
Name: signup_app, dtype: int64 

signup_flow 
False    275547
Name: signup_flow, dtype: int64 

signup_m

 From above we can see that there are so many NaN in some of the columns, like <font color = 'red'>age</font> and <font color = 'red'>date_first_booking</font>. Then how should we cope with these NaN data? If we simply replace them with 99999, I think it's not a appropriate way to do so.

 And after this, we know that there is no data about date_first_booking in test data, so what we have here in date_first_booking will not work in the test data, even worse, it will create a massive number of NaN data for us which can really mess with the model and prediction I think.<br />After all, there're only 60k rows in test data, and there're 186k NaN in full data. It could be a huge negtive effect for the model.

 Some null in first_affiliate_tracked, but just 6k of them, and 269k are valid value, so just ignore the null maybe.

### Group Original variables

In [22]:

target = ['country_destination']
id_vars = ['id']
num_vars_org = ['age'
#                 , 'signup_flow'
               ]
datetime_vars_org = ['date_account_created', 'date_first_booking', 'timestamp_first_active']
cat_vars_org = ['affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'first_browser', \
               'first_device_type', 'gender', 'language', 'signup_app', 'signup_method'
               , 'signup_flow'
               ]

## Step 2 Feature Engineering

## Numerical variables

### new numerical variables from date time

In [23]:
# ---------- created ----------
fullData['created_year'] = fullData['date_account_created'].dt.year
fullData['created_month'] = fullData['date_account_created'].dt.month
fullData['created_day'] = fullData['date_account_created'].dt.day
fullData['created_dayofweek'] = fullData['date_account_created'].dt.dayofweek
fullData['created_dayofyear'] = fullData['date_account_created'].dt.dayofyear
fullData['created_weekofyear'] = fullData['date_account_created'].dt.weekofyear

# ---------- active ----------
fullData['first_active_year'] = fullData['timestamp_first_active'].dt.year
fullData['first_active_month'] = fullData['timestamp_first_active'].dt.month
fullData['first_active_day'] = fullData['timestamp_first_active'].dt.day
fullData['first_active_dayofweek'] = fullData['timestamp_first_active'].dt.dayofweek
fullData['first_active_dayofyear'] = fullData['timestamp_first_active'].dt.dayofyear
fullData['first_active_weekofyear'] = fullData['timestamp_first_active'].dt.weekofyear

# ---------- booking ----------
# fullData['first_booking_year'] = fullData['date_first_booking'].dt.year
# fullData['first_booking_month'] = fullData['date_first_booking'].dt.month
# fullData['first_booking_day'] = fullData['date_first_booking'].dt.day
# fullData['first_booking_dayofweek'] = fullData['date_first_booking'].dt.dayofweek
# fullData['first_booking_dayofyear'] = fullData['date_first_booking'].dt.dayofyear
# fullData['first_booking_weekofyear'] = fullData['date_first_booking'].dt.weekofyear



num_datetime_vars = ['created_year', 'created_month', 'created_day', 'created_dayofweek', 'created_dayofyear', \
                    'created_weekofyear', \
                    'first_active_year', 'first_active_month', 'first_active_day', 'first_active_dayofweek', \
                    'first_active_dayofyear', 'first_active_weekofyear', \
#                     'first_booking_year', 'first_booking_month', 'first_booking_day', 'first_booking_dayofweek', \
#                     'first_booking_dayofyear', 'first_booking_weekofyear'
                    ]

### Numerical variables <font color = 'red'>Interaction</font> among date and time variables

In [24]:
# ---------- active-created ----------
fullData['diff_created_firstActive_year'] = fullData['first_active_year'] - fullData['created_year']
fullData['diff_created_firstActive_month'] = fullData['first_active_month'] - fullData['created_month']
fullData['diff_created_firstActive_day'] = fullData['first_active_day'] - fullData['created_day']

# ---------- booking-active ----------
# fullData['diff_firstActive_booking_year'] = fullData['first_booking_year'] - fullData['first_active_year']
# fullData['diff_firstActive_booking_month'] = fullData['first_booking_month'] - fullData['first_active_month']
# fullData['diff_firstActive_booking_day'] = fullData['first_booking_day'] - fullData['first_active_day']

num_datetime_vars.append('diff_created_firstActive_year')
num_datetime_vars.append('diff_created_firstActive_month')
num_datetime_vars.append('diff_created_firstActive_day')
# num_datetime_vars.append('diff_firstActive_booking_year')
# num_datetime_vars.append('diff_firstActive_booking_month')
# num_datetime_vars.append('diff_firstActive_booking_day')

## Categorical variables

### One-Hot encoding some of the categorical data

In [25]:
dummiedMergedCoutriesTrainUses = pd.get_dummies(fullData, columns = cat_vars_org)

In [26]:
for header in dummiedMergedCoutriesTrainUses:
    print header

age
country_destination
date_account_created
date_first_booking
id
timestamp_first_active
created_year
created_month
created_day
created_dayofweek
created_dayofyear
created_weekofyear
first_active_year
first_active_month
first_active_day
first_active_dayofweek
first_active_dayofyear
first_active_weekofyear
diff_created_firstActive_year
diff_created_firstActive_month
diff_created_firstActive_day
affiliate_channel_api
affiliate_channel_content
affiliate_channel_direct
affiliate_channel_other
affiliate_channel_remarketing
affiliate_channel_sem-brand
affiliate_channel_sem-non-brand
affiliate_channel_seo
affiliate_provider_baidu
affiliate_provider_bing
affiliate_provider_craigslist
affiliate_provider_daum
affiliate_provider_direct
affiliate_provider_email-marketing
affiliate_provider_facebook
affiliate_provider_facebook-open-graph
affiliate_provider_google
affiliate_provider_gsp
affiliate_provider_meetup
affiliate_provider_naver
affiliate_provider_other
affiliate_provider_padmapper
affiliat

In [27]:
dummiedMergedCoutriesTrainUses.drop('date_first_booking', axis = 1, inplace = True)

In [28]:
for header in dummiedMergedCoutriesTrainUses:
    print header

age
country_destination
date_account_created
id
timestamp_first_active
created_year
created_month
created_day
created_dayofweek
created_dayofyear
created_weekofyear
first_active_year
first_active_month
first_active_day
first_active_dayofweek
first_active_dayofyear
first_active_weekofyear
diff_created_firstActive_year
diff_created_firstActive_month
diff_created_firstActive_day
affiliate_channel_api
affiliate_channel_content
affiliate_channel_direct
affiliate_channel_other
affiliate_channel_remarketing
affiliate_channel_sem-brand
affiliate_channel_sem-non-brand
affiliate_channel_seo
affiliate_provider_baidu
affiliate_provider_bing
affiliate_provider_craigslist
affiliate_provider_daum
affiliate_provider_direct
affiliate_provider_email-marketing
affiliate_provider_facebook
affiliate_provider_facebook-open-graph
affiliate_provider_google
affiliate_provider_gsp
affiliate_provider_meetup
affiliate_provider_naver
affiliate_provider_other
affiliate_provider_padmapper
affiliate_provider_vast
aff

### Label encoding the target

In [29]:
LE = preprocessing.LabelEncoder()
labels_transed = LE.fit_transform(dummiedMergedCoutriesTrainUses['country_destination'][:train_size])
    

readyToLearnDataSet = pd.DataFrame(dummiedMergedCoutriesTrainUses)

## Step 3 Train the model

1. realized that the model cannot learn datetime data type
2. 'destination_language' is a mysterious variable that nobody can do things with it

In [30]:
train_X = readyToLearnDataSet.drop(['country_destination', 'id',\
                                  'date_account_created',\
                                  'timestamp_first_active' ], axis = 1)[:train_size]

train_y = labels_transed.reshape(-1,1)


test_X = readyToLearnDataSet.drop(['country_destination', 'id',\
                                  'date_account_created',\
                                  'timestamp_first_active' ], axis = 1)[train_size:]

test_y = readyToLearnDataSet[['country_destination']][train_size:]

In [31]:
print train_X.shape , 'train_X shape'
print train_y.shape , 'train_y shape'
print test_X.shape , 'test_X shape'
print test_y.shape , 'test_y shape'

# print test_y.country_destination.value_counts()

(213451, 169) train_X shape
(213451, 1) train_y shape
(62096, 169) test_X shape
(62096, 1) test_y shape


In [32]:
for header11 in readyToLearnDataSet:
    print readyToLearnDataSet[header11].isnull().value_counts()

False    275547
Name: age, dtype: int64
False    213451
True      62096
Name: country_destination, dtype: int64
False    275547
Name: date_account_created, dtype: int64
False    275547
Name: id, dtype: int64
False    275547
Name: timestamp_first_active, dtype: int64
False    275547
Name: created_year, dtype: int64
False    275547
Name: created_month, dtype: int64
False    275547
Name: created_day, dtype: int64
False    275547
Name: created_dayofweek, dtype: int64
False    275547
Name: created_dayofyear, dtype: int64
False    275547
Name: created_weekofyear, dtype: int64
False    275547
Name: first_active_year, dtype: int64
False    275547
Name: first_active_month, dtype: int64
False    275547
Name: first_active_day, dtype: int64
False    275547
Name: first_active_dayofweek, dtype: int64
False    275547
Name: first_active_dayofyear, dtype: int64
False    275547
Name: first_active_weekofyear, dtype: int64
False    275547
Name: diff_created_firstActive_year, dtype: int64
False    275547
N

### There is no <font color='red'>'Null'</font> value in this data set now

### Implement Scorer

In [33]:
# ---------------------------- copy from someone else ----------------------------
# https://github.com/davidgasquez/kaggle-airbnb/blob/master/kairbnb/metrics.py
# https://www.kaggle.com/davidgasquez/airbnb-recruiting-new-user-bookings/ndcg-scorer/code

"""Metrics to compute the model performance."""

import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import make_scorer


def dcg_score(y_true, y_score, k=5):
    """Discounted cumulative gain (DCG) at rank K.

    Parameters
    ----------
    y_true : array, shape = [n_samples]
        Ground truth (true relevance labels).
    y_score : array, shape = [n_samples, n_classes]
        Predicted scores.
    k : int
        Rank.

    Returns
    -------
    score : float
    """
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)


def ndcg_score(ground_truth, predictions, k=5):
    """Normalized discounted cumulative gain (NDCG) at rank K.

    Normalized Discounted Cumulative Gain (NDCG) measures the performance of a
    recommendation system based on the graded relevance of the recommended
    entities. It varies from 0.0 to 1.0, with 1.0 representing the ideal
    ranking of the entities.

    Parameters
    ----------
    ground_truth : array, shape = [n_samples]
        Ground truth (true labels represended as integers).
    predictions : array, shape = [n_samples, n_classes]
        Predicted probabilities.
    k : int
        Rank.

    Returns
    -------
    score : float

    Example
    -------
    >>> ground_truth = [1, 0, 2]
    >>> predictions = [[0.15, 0.55, 0.2], [0.7, 0.2, 0.1], [0.06, 0.04, 0.9]]
    >>> score = ndcg_score(ground_truth, predictions, k=2)
    1.0
    >>> predictions = [[0.9, 0.5, 0.8], [0.7, 0.2, 0.1], [0.06, 0.04, 0.9]]
    >>> score = ndcg_score(ground_truth, predictions, k=2)
    0.6666666666
    """
    lb = LabelBinarizer()
    lb.fit(range(len(predictions) + 1))
    T = lb.transform(ground_truth)

    scores = []

    # Iterate over each y_true and compute the DCG score
    for y_true, y_score in zip(T, predictions):
        actual = dcg_score(y_true, y_score, k)
        best = dcg_score(y_true, y_true, k)
        score = float(actual) / float(best)
        scores.append(score)

    return np.mean(scores)


# NDCG Scorer function
ndcg_scorer = make_scorer(ndcg_score, needs_proba=True, k=5)


In [34]:
clf_xgb = xgb.XGBClassifier(learning_rate = 0.3
                  , n_estimators = 25
                  , max_depth = 6
                  , min_child_weight = 1
                  , subsample = 0.5
                  , colsample_bytree = 0.5
                  , gamma = 1
                  , seed = 1234
                  , nthread = -1
                  , objective='multi:softprob'
                  )
clf_xgb.fit(train_X, train_y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=1, learning_rate=0.3, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=25, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=1234, silent=True, subsample=0.5)

In [35]:
preds = clf_xgb.predict_proba(test_X)
print pd.DataFrame(preds)
# print pd.DataFrame(LE.inverse_transform(preds))

             0         1         2         3         4         5         6   \
0      0.002320  0.008866  0.003131  0.009130  0.024038  0.005922  0.006301   
1      0.001227  0.003702  0.002116  0.003406  0.006146  0.003397  0.004107   
2      0.001663  0.005109  0.007614  0.010596  0.009887  0.006460  0.009074   
3      0.001603  0.004348  0.008125  0.009490  0.009530  0.006227  0.008747   
4      0.001907  0.005501  0.003895  0.008014  0.014404  0.009445  0.010459   
5      0.002963  0.011551  0.007303  0.013732  0.027112  0.016503  0.021612   
6      0.003287  0.009839  0.006471  0.008893  0.018353  0.008912  0.016055   
7      0.001913  0.006315  0.003932  0.008092  0.014184  0.008725  0.011135   
8      0.001913  0.006315  0.003932  0.008092  0.014184  0.008725  0.011135   
9      0.001680  0.004824  0.007254  0.006090  0.011431  0.005861  0.007441   
10     0.001903  0.005820  0.006229  0.007831  0.011738  0.008236  0.010838   
11     0.003114  0.007276  0.003989  0.012011  0.019

In [36]:
id_test = readyToLearnDataSet['id'][train_size:]

## Get the top 5 probabilities 

In [37]:
# Taking the 5 classes with highest probabilities
ids = []
cts = []
for i in range(len(id_test)):
    idx = id_test[i]
    ids += [idx] * 5
    cts += LE.inverse_transform(np.argsort(preds[i])[::-1])[:5].tolist()
    

# Write csv files as submission
sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
sub.to_csv('sub_201710041410.csv',index=False)

## Feature importance

In [130]:
full_vars = []
for header in train_X:
    full_vars.append(header)




feature_importance = pd.DataFrame(sorted(zip(full_vars,clf_xgb.feature_importances_)
                          , key=lambda x: x[1], reverse = True),columns=['feature_name','importance']) 

print (feature_importance.query('importance>0'))

                           feature_name  importance
0                     created_dayofyear    0.106512
1                                   age    0.098527
2                first_active_dayofyear    0.083905
3                           created_day    0.063660
4                    created_weekofyear    0.040940
5                     created_dayofweek    0.037341
6                          created_year    0.034417
7                      first_active_day    0.033180
8                first_active_dayofweek    0.024857
9               first_active_weekofyear    0.024857
10                    first_active_year    0.024069
11        first_device_type_Mac Desktop    0.018221
12    first_affiliate_tracked_untracked    0.017658
13                  signup_method_basic    0.015971
14                        gender_FEMALE    0.014959
15                     gender_-unknown-    0.014172
16                          gender_MALE    0.013384
17                        signup_flow_3    0.013159
18          

### Check the prediction

In [12]:
print y
print le.inverse_transform(y)

[ 7  7 10 ...,  7  7  7]
['NDF' 'NDF' 'US' ..., 'NDF' 'NDF' 'NDF']
